 ## Carga y Transformación de los archivos

 Tomaremos los archivos Excel de **homicidios.xlsx** &  **lesiones.xlsx**, haremos las transformaciones y lo dejaremos listo como CSV para su carga en SQL
 

 Como libreria principal utilizaremos Pandas, aunque será necesario tener instalado *openpyxl*
 será necesario utulizar re para cierto paso del dataframe **comunas**

In [1]:
import pandas as pd
import re

para importar el dataframe **comunas** será necesario abrir el excel de *pob_comunas.xlsx*, irnos hasta la hoja "Cuadro 1.1"  y copiar todo el contenido del cuadro

In [2]:
rut_hom = r'C:\Users\OS\Desktop\Proyecto Buenos aires\Data\raw\homicidios.xlsx'
rut_les = r'C:\Users\OS\Desktop\Proyecto Buenos aires\Data\raw\lesiones.xlsx'
homicidios = pd.read_excel(rut_hom,sheet_name='HECHOS')
victimas = pd.read_excel(rut_hom, sheet_name='VICTIMAS')
lesiones = pd.read_excel(rut_les,sheet_name='HECHOS')
comunas = pd.read_clipboard()

# comenzamos por Homicidios

Observamos que cuenta con varias columnas que son redundantes, y pocos valores nulos
esto de los valores nulos es confuso, hay muchos valores 'SD' (sin datos)

In [3]:
homicidios.isna().sum()


ID                         0
N_VICTIMAS                 0
FECHA                      0
AAAA                       0
MM                         0
DD                         0
HORA                       0
HH                         0
LUGAR_DEL_HECHO            0
TIPO_DE_CALLE              0
Calle                      1
Altura                   567
Cruce                    171
Dirección Normalizada      8
COMUNA                     0
XY (CABA)                  0
pos x                      0
pos y                      0
PARTICIPANTES              0
VICTIMA                    0
ACUSADO                    0
dtype: int64

In [4]:
homicidios.head(2)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS


la columna **lugar del hecho** no cuenta con valores nulos, asi que la usaremos para crear una nueva columna **LUGAR_DEL_HECHO_NORMALIZADO** Y luego quitaremos las columnas que no interesan

In [5]:
homicidios['LUGAR_DEL_HECHO_NORMALIZADO'] = homicidios['LUGAR_DEL_HECHO'].str.upper() \
    .str.replace(' Y ', ' & ') \
    .str.replace(r'\bAV(ENIDA)?\b', '') \
    .str.replace(r'\s+', ' ') \
    .str.strip()

In [6]:
columnas = ['LUGAR_DEL_HECHO', 'Dirección Normalizada','AAAA','MM','Altura','DD','PARTICIPANTES', 'XY (CABA)', 'Cruce']
homicidios = homicidios.drop(columns=columnas)

In [7]:
columnas = homicidios.columns.tolist()
indice_calle = columnas.index('Calle')
columnas.insert(indice_calle + 1, 'LUGAR_DEL_HECHO_NORMALIZADO')
homicidios = homicidios[columnas]

Obsevamos los formatos de cada columna y nos damos cuenta lo siguiente:

**ID:**
Tiene valores numericos pero con un caracter '-' que lo quitaremos para poder volverlo numerico

**HORA:** 
Tiene falores SD que seran necesario transformar para convertirlo a formato hora en SQL

**HH:**
Contiene valores tipo SD que  impiden su paso a formato Entero

**pos x & pos y:**
representan longitud y latitud respectivamente, seran tratados y conveertidos a float para un mejor manejo

**VICTIMA,ACUSADO,CALLE:**
sus valores 'SD' seran pasados a DESCONOCIDO


In [8]:
homicidios = homicidios.loc[:, ~homicidios.columns.duplicated()]
homicidios.dtypes

ID                                     object
N_VICTIMAS                              int64
FECHA                          datetime64[ns]
HORA                                   object
HH                                     object
TIPO_DE_CALLE                          object
Calle                                  object
LUGAR_DEL_HECHO_NORMALIZADO            object
COMUNA                                  int64
pos x                                  object
pos y                                  object
VICTIMA                                object
ACUSADO                                object
dtype: object

In [9]:
homicidios['ID'] = homicidios['ID'].str.replace('-', '')
homicidios['ID'] = pd.to_numeric(homicidios['ID'])
homicidios['HH'] = pd.to_numeric(homicidios['HH'], errors='coerce')
homicidios.rename(columns={'pos x': 'longitud',
                           'pos y': 'Latitud'}, inplace= True)

In [10]:
homicidios.isnull().sum()

ID                             0
N_VICTIMAS                     0
FECHA                          0
HORA                           0
HH                             1
TIPO_DE_CALLE                  0
Calle                          1
LUGAR_DEL_HECHO_NORMALIZADO    0
COMUNA                         0
longitud                       0
Latitud                        0
VICTIMA                        0
ACUSADO                        0
dtype: int64

In [11]:
null_HH = homicidios.loc[homicidios['HH'].isnull()]
null_calle = homicidios.loc[homicidios['Calle'].isnull()]
print(null_calle)
print(null_HH)

           ID  N_VICTIMAS      FECHA      HORA    HH TIPO_DE_CALLE Calle  \
119  20160151           1 2016-11-18  20:35:00  20.0         CALLE   NaN   

    LUGAR_DEL_HECHO_NORMALIZADO  COMUNA longitud Latitud VICTIMA ACUSADO  
119                          SD       0        .       .  PEATON      SD  
           ID  N_VICTIMAS      FECHA HORA  HH TIPO_DE_CALLE           Calle  \
518  20190103           1 2019-12-18   SD NaN      GRAL PAZ  PAZ, GRAL. AV.   

    LUGAR_DEL_HECHO_NORMALIZADO  COMUNA      longitud       Latitud VICTIMA  \
518     PAZ, GRAL. AV. & GRIVEO      11  -58.52169422  -34.59471640    MOTO   

    ACUSADO  
518    MOTO  


In [12]:
homicidios.loc[homicidios['HH'].isnull(), 'HH'] = '00'
homicidios['HH'] = pd.to_numeric(homicidios['HH'])
homicidios.loc[homicidios['Calle'].isnull(), 'Calle'] = 'DESCONOCIDO'


C:\Users\OS\AppData\Local\Temp\ipykernel_18836\1592614858.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '00' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  homicidios.loc[homicidios['HH'].isnull(), 'HH'] = '00'


In [13]:
homicidios.loc[homicidios['HORA'] == 'SD', 'HORA'] = '00:00:00'
homicidios.loc[homicidios['VICTIMA'] == 'SD', 'VICTIMA'] = 'DESCONOCIDO'
homicidios.loc[homicidios['ACUSADO'] == 'SD', 'ACUSADO'] = 'DESCONOCIDO'

In [14]:
homicidios['Latitud'] = homicidios['Latitud'].str.replace(',', '.')
homicidios['longitud'] = homicidios['longitud'].str.replace(',', '.')
homicidios = homicidios[homicidios['Latitud'] != '.']
homicidios = homicidios[homicidios['longitud'] != '.']
homicidios['Latitud'] = homicidios['Latitud'].astype(float)
homicidios['longitud'] = homicidios['longitud'].astype(float)

In [15]:
homicidios['HH'] = homicidios['HH'].astype(int)

## Lesiones

este archivo requiere un trabajo mas extenso

tiene una cantidad de nulos y valores 'SD', y columnas que hay que tratar, eliminar o modificar para crear un archivo listo para analizar y cargar a SQL

In [16]:
lesiones['direc_normalizada'] = lesiones['otra_direccion'].apply(lambda x: x.lower() if pd.notnull(x) else x)


Las columnas:

**'aaaa','mm','dd'**

**'calle','altura','geocodificacion_CABA'**

**'participantes','moto','auto','transporte_publico'**

**'camion','ciclista','direccion_normalizada', 'cruce','otra_direccion'**

seran quitadas puesto no aportan nada relevante

In [17]:
columnas = ['aaaa','mm','dd','calle','altura','geocodificacion_CABA','participantes','moto','auto','transporte_publico','camion','ciclista','direccion_normalizada', 'cruce','otra_direccion']
lesiones = lesiones.drop(columns=columnas)

Se quitan los caracteres no numericos de **id**

se pasa a formato datetime a la columna **fecha**

se inserta la columna **direc_normalizada** antes de *longitud* y se le cambian los valores 'SD' a 'sin datos'

se toman los valores nulos, SD y 'No especificada' (hay uno solo de este ultimo) a 0, valor numerico que queda como indicador de desconocido

In [18]:
lesiones['id'] = lesiones['id'].str.replace(r'\D', '', regex=True)
lesiones['fecha'] = pd.to_datetime(lesiones['fecha']).dt.strftime('%Y-%m-%d')
columnas = lesiones.columns.tolist()
indice_longitud = columnas.index('longitud')
columnas.insert(indice_longitud, 'direc_normalizada')
lesiones = lesiones[columnas]
lesiones['direc_normalizada'] = lesiones['direc_normalizada'].replace('sd', 'sin datos', regex=True)
lesiones = lesiones.loc[:,~lesiones.columns.duplicated()]

In [19]:
lesiones['comuna'] = lesiones['comuna'].replace('SD', 0)
lesiones['comuna'] = lesiones['comuna'].replace('No Especificada', 0)
lesiones['comuna'] = lesiones['comuna'].fillna(0)
lesiones['comuna'] = pd.to_numeric(lesiones['comuna'])

C:\Users\OS\AppData\Local\Temp\ipykernel_18836\3351253537.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  lesiones['comuna'] = lesiones['comuna'].replace('No Especificada', 0)


Le daremos formato de hora a la columna *hora*

la columna **tipo_Calle** no cuenta con valores nulos pero si muchos tipo 'SD' los cuales transformaremos en:

- AUTOPISTA
- AVENIDA
- CALLE
- GRAL PAZ

esto lo haremos en base a los valores de 'direc_normalizada'

In [20]:
def convertir_formato_hora(hora):
    hora_datetime = pd.to_datetime(hora, format='%H:%M:%S', errors='coerce')
    hora_24h = hora_datetime.strftime('%H:%M:%S') if not pd.isnull(hora_datetime) else None
    return hora_24h
lesiones['hora'] = lesiones['hora'].apply(convertir_formato_hora)

In [21]:
lesiones.loc[lesiones['direc_normalizada'].str.contains('autopista', case=False), 'tipo_calle'] = 'AUTOPISTA'
lesiones.loc[lesiones['direc_normalizada'].str.contains('av.', case=False), 'tipo_calle'] = 'AVENIDA'
lesiones.loc[lesiones['direc_normalizada'].str.contains('calle', case=False), 'tipo_calle'] = 'CALLE'
lesiones.loc[lesiones['direc_normalizada'] == 'paz, gral. av.', 'tipo_calle'] = 'GRAL PAZ'
lesiones.loc[lesiones['tipo_calle'] == 'SD', 'tipo_calle'] = 'CALLE'


para las columnas de **latitud** y **longitud** llenaremos los valores nulos con 0 y luego reemplazaramos los outliers y los valores iguales a 0 con la media de cada columna


In [22]:
lesiones['latutid'] =lesiones['latutid'].fillna(00.00000)
lesiones['longitud'] =lesiones['longitud'].fillna(00.00000)
lesiones['latutid'] =lesiones['latutid'].replace('SD',00.00000)
lesiones['longitud'] =lesiones['longitud'].replace('SD',00.00000)

In [23]:
lesiones.rename(columns={'latutid': 'latitud'}, inplace=True)
lesiones['latitud'] = lesiones['latitud'].astype(float)
lesiones['longitud'] = lesiones['longitud'].astype(float)
lesiones['latitud'] = lesiones['latitud'].round(8)
lesiones['longitud'] = lesiones['longitud'].round(8)
lesiones.loc[lesiones['latitud'] == 0, 'latitud'] = lesiones['latitud'].replace(0, lesiones.loc[lesiones['latitud'] != 0, 'latitud'].mean()).round(8)
lesiones.loc[lesiones['longitud'] == 0, 'longitud'] = lesiones['longitud'].replace(0, lesiones.loc[lesiones['longitud'] != 0, 'longitud'].mean()).round(8)
pd.options.display.float_format = '{:.8f}'.format
lesiones.loc[23784, 'longitud'] = -58.29963273
lesiones.loc[23784, 'latitud'] = -34.72890716

In [24]:
lesiones['comuna'] = lesiones['comuna'].astype(int)
lesiones['n_victimas'] = lesiones['n_victimas'].astype(int)
lesiones.loc[lesiones['gravedad'] == 'SD', 'gravedad'] = 'Sin Gravedad'
lesiones.loc[lesiones['acusado'] == 'SD', 'acusado'] = 'desconocido'
lesiones.loc[lesiones['victima'] == 'SD', 'victima'] = 'desconocido'

In [25]:
# errores al cargar long y lat
long_med_f = lesiones.loc[(lesiones['longitud'] > -180.0) & (lesiones['longitud'] < 180.0), 'longitud']
lat_med_f = lesiones.loc[(lesiones['latitud'] > -180.0) & (lesiones['latitud'] < 180.0), 'latitud']
long_med = round(long_med_f.mean(),8)
lat_med = round(lat_med_f.mean(),8)
lesiones.loc[lesiones['longitud'] < -180.0, 'longitud'] = long_med
lesiones.loc[lesiones['longitud'] > 180.0, 'longitud'] = long_med
lesiones.loc[lesiones['latitud'] < -180.0, 'latitud'] = lat_med
lesiones.loc[lesiones['latitud'] > 180.0, 'latitud'] = lat_med

Reemplazaremos los valores nulos en **franja_hora** con la media de esta misma columna, para luego rellenar los datos vacios en la columna **hora** con su valor consecuente de **franja_hora**

In [26]:
# manejo de nulos en hora y despues se llenara franja_hora
lesiones['franja_hora'] = lesiones['franja_hora'].replace(['sd','SD'], pd.NA)
med = round(lesiones['franja_hora'].mean(skipna= True))
lesiones['franja_hora'].fillna(med, inplace= True)
lesiones['hora'] = lesiones['hora'].fillna(lesiones['franja_hora'].astype(str).str[:2] + ':00:00')

C:\Users\OS\AppData\Local\Temp\ipykernel_18836\3302146950.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  lesiones['franja_hora'].fillna(med, inplace= True)
C:\Users\OS\AppData\Local\Temp\ipykernel_18836\3302146950.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  lesiones['franja_hora'].fillna(med, inpla

comenzaremos con el dataframe 
# victima

In [27]:
victimas.head()

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,SD
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01 00:00:00


quitamos las columnas que no nos brindan informacion util

In [28]:
columnas = ['AAAA','MM','DD']
victimas.drop(columns= columnas,inplace= True)

tratermoss los valores **"SD"**

In [29]:
victimas.eq("SD").sum()

ID_hecho                0
FECHA                   0
ROL                    11
VICTIMA                 9
SEXO                    6
EDAD                   53
FECHA_FALLECIMIENTO    68
dtype: int64

In [30]:

victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_hecho             717 non-null    object        
 1   FECHA                717 non-null    datetime64[ns]
 2   ROL                  717 non-null    object        
 3   VICTIMA              717 non-null    object        
 4   SEXO                 717 non-null    object        
 5   EDAD                 717 non-null    object        
 6   FECHA_FALLECIMIENTO  717 non-null    object        
dtypes: datetime64[ns](1), object(6)
memory usage: 39.3+ KB


haremos las transformaciones pertinentes

- **ID_hecho** se quitan los "-" y se vuelve a tipo de dato entero
- **FECHA** se transforma su formato a datetime
- **ROL** se cambian los valores de ***SD*** y ***pasajero_acompañante*** a "DESCONOCIDO" y "PASAJERO" respectivamente
- **VICTIMA** se cambian los valores de ***SD*** a "DESCONOCIDO"
- **EDAD** se reemplazan los valores ***SD*** por la media de la columna y luego se pasa a valor entero
- **FECHA_FALLECIMIENTO** se quitan los nulos puesto que se interpeta que el accidentado no murió

In [31]:
victimas['ID_hecho'] = victimas['ID_hecho'].str.replace('-', '')
victimas['ID_hecho']=victimas['ID_hecho'].astype(int)
victimas['FECHA'] = pd.to_datetime(victimas['FECHA'])
victimas['ROL'] = victimas['ROL'].replace({'SD': 'DESCONOCIDO', 'PASAJERO_ACOMPAÑANTE': 'PASAJERO'})
victimas['VICTIMA'] = victimas['VICTIMA'].replace({'SD': 'DESCONOCIDO'})
victimas['SEXO'] = victimas['SEXO'].replace({'SD': 'DESCONOCIDO'})
victimas['EDAD'] = pd.to_numeric(victimas['EDAD'], errors='coerce',) 
edad_media = victimas['EDAD'].mean()
victimas['EDAD'] = victimas['EDAD'].fillna(edad_media)
victimas['EDAD'] = victimas['EDAD'].astype(int)
victimas = victimas[victimas['FECHA_FALLECIMIENTO'] != 'SD']


se unen los dataframes de ***Homicidios*** y ***victimas***

quedando merged_df como dataframe que contiene todas las columnas de Homicidios y la columnas **'SEXO', 'EDAD', 'ROL'** como columnas agregadas

la union se hace a traves de
## homicidios['ID'] == victimas['ID_hecho']

In [32]:
merged_df = pd.merge(homicidios, victimas[['ID_hecho', 'SEXO', 'EDAD', 'ROL']], left_on='ID', right_on='ID_hecho', how='left')
merged_df.drop(columns=['ID_hecho'], inplace=True)
merged_df.rename(columns={'ROL': 'ROL_VICTIMA'}, inplace=True)
merged_df.head()

,ID,N_VICTIMAS,FECHA,HORA,HH,TIPO_DE_CALLE,Calle,LUGAR_DEL_HECHO_NORMALIZADO,COMUNA,longitud,Latitud,VICTIMA,ACUSADO,SEXO,EDAD,ROL_VICTIMA
0,20160001,1,2016-01-01,04:00:00,4,AVENIDA,PIEDRA BUENA AV.,AV PIEDRA BUENA & AV FERNANDEZ DE LA CRUZ,8,-58.47533969,-34.68757022,MOTO,AUTO,MASCULINO,19.00000000,CONDUCTOR
1,20160002,1,2016-01-02,01:15:00,1,GRAL PAZ,"PAZ, GRAL. AV.",AV GRAL PAZ & AV DE LOS CORRALES,9,-58.50877521,-34.66977709,AUTO,PASAJEROS,MASCULINO,70.00000000,CONDUCTOR
2,20160003,1,2016-01-03,07:00:00,7,AVENIDA,ENTRE RIOS AV.,AV ENTRE RIOS 2034,1,-58.39040293,-34.63189362,MOTO,AUTO,MASCULINO,30.00000000,CONDUCTOR
3,20160004,1,2016-01-10,00:00:00,0,AVENIDA,LARRAZABAL AV.,AV LARRAZABAL & GRAL VILLEGAS CONRADO,8,-58.46503904,-34.68092974,MOTO,DESCONOCIDO,NaN,NaN,NaN
4,20160005,1,2016-01-21,05:20:00,5,AVENIDA,SAN JUAN AV.,AV SAN JUAN & PRESIDENTE LUIS SAENZ PEÑA,1,-58.38718297,-34.62246630,MOTO,PASAJEROS,MASCULINO,29.00000000,CONDUCTOR


Volvemos a haceer la transformacion en edad pero esta vez con los nulos para las filas que no tuvieron union

para las columnas de **SEXO, EDAD** que tienen la misma condicion, se rellena el nulo con "DESCONOCIDO"

In [47]:
edad_media = merged_df['EDAD'].mean()
merged_df['EDAD'] = merged_df['EDAD'].fillna(edad_media)
merged_df['EDAD'] = merged_df['EDAD'].astype(int)
merged_df['SEXO'] = merged_df['SEXO'].fillna('DESCONOCIDO')
merged_df['ROL_VICTIMA'] = merged_df['ROL_VICTIMA'].fillna('DESCONOCIDO')
merged_df.drop_duplicates(subset = ['ID'],keep='first',inplace=True)
merged_df.head()


,ID,N_VICTIMAS,FECHA,HORA,HH,TIPO_DE_CALLE,Calle,LUGAR_DEL_HECHO_NORMALIZADO,COMUNA,longitud,Latitud,VICTIMA,ACUSADO,SEXO,EDAD,ROL_VICTIMA
0,20160001,1,2016-01-01,04:00:00,4,AVENIDA,PIEDRA BUENA AV.,AV PIEDRA BUENA & AV FERNANDEZ DE LA CRUZ,8,-58.47533969,-34.68757022,MOTO,AUTO,MASCULINO,19,CONDUCTOR
1,20160002,1,2016-01-02,01:15:00,1,GRAL PAZ,"PAZ, GRAL. AV.",AV GRAL PAZ & AV DE LOS CORRALES,9,-58.50877521,-34.66977709,AUTO,PASAJEROS,MASCULINO,70,CONDUCTOR
2,20160003,1,2016-01-03,07:00:00,7,AVENIDA,ENTRE RIOS AV.,AV ENTRE RIOS 2034,1,-58.39040293,-34.63189362,MOTO,AUTO,MASCULINO,30,CONDUCTOR
3,20160004,1,2016-01-10,00:00:00,0,AVENIDA,LARRAZABAL AV.,AV LARRAZABAL & GRAL VILLEGAS CONRADO,8,-58.46503904,-34.68092974,MOTO,DESCONOCIDO,DESCONOCIDO,42,DESCONOCIDO
4,20160005,1,2016-01-21,05:20:00,5,AVENIDA,SAN JUAN AV.,AV SAN JUAN & PRESIDENTE LUIS SAENZ PEÑA,1,-58.38718297,-34.62246630,MOTO,PASAJEROS,MASCULINO,29,CONDUCTOR


si la copia del dataframe comunas se realizo bien, deberiamms tener algo como lo siguiente:

In [34]:
comunas.head()

,Código,Comuna,2010,2022,Variación absoluta,Variación relativa (%)
0,NaN,NaN,NaN,NaN,NaN,NaN
1,2007.00000000,Comuna 1,205.88600000,223.55400000,17.66800000,"8,6"
2,2014.00000000,Comuna 2,157.93200000,161.64500000,3.71300000,"2,4"
3,2021.00000000,Comuna 3,187.53700000,196.24000000,8.70300000,"4,6"
4,2028.00000000,Comuna 4,218.24500000,229.24000000,10.99500000,"5,0"


Nos quedaremos solamente con lo importante, puesto que la idea de este dataframe es brindar algo de contexto en las comunas

In [35]:
comunas = comunas.dropna(subset=['Comuna'])
columnas = ['Código','2010','Variación absoluta', 'Variación relativa (%)']
comunas = comunas.drop(columns= columnas)
comunas['Comuna'] = comunas['Comuna']
comunas['Comuna'] = comunas['Comuna'].apply(lambda x: re.findall(r'\d+', str(x))[0] if re.findall(r'\d+', str(x)) else None)
comunas['2022']= comunas['2022'].astype(float).map('{:.3f}'.format)


In [41]:
comunas.head()

,Comuna,2022,Cantidad de Localidades
0,0,0,0
1,1,223.554,6
2,2,161.645,1
3,3,196.240,2
4,4,229.240,4


Tenemo la poblacion total de cada comuna para 2022 pero tambien agregaremos cuantas localidades conforman cada comuna

esta informacion esta bajo el cuadro de donde sacamos el dataframe y se ve asi:

In [37]:
localidades_por_comuna = {
    '1': ['Retiro', 'San Nicolás', 'Puerto Madero', 'San Telmo', 'Montserrat', 'Constitución'],
    '2': ['Recoleta'],
    '3': ['Balvanera', 'San Cristóbal'],
    '4': ['La Boca', 'Barracas', 'Parque Patricios', 'Nueva Pompeya'],
    '5': ['Almagro', 'Boedo'],
    '6': ['Caballito'],
    '7': ['Flores', 'Parque Chacabuco'],
    '8': ['Villa Soldati', 'Villa Riachuelo', 'Villa Lugano'],
    '9': ['Liniers', 'Mataderos', 'Parque Avellaneda'],
    '10': ['Villa Real', 'Monte Castro', 'Versalles', 'Floresta', 'Vélez Sarsfield', 'Villa Luro'],
    '11': ['Villa General Mitre', 'Villa Devoto', 'Villa del Parque', 'Villa Santa Rita'],
    '12': ['Coghlan', 'Saavedra', 'Villa Urquiza', 'Villa Pueyrredón'],
    '13': ['Núñez', 'Belgrano', 'Colegiales'],
    '14': ['Palermo'],
    '15': ['Chacarita', 'Villa Crespo', 'La Paternal', 'Villa Ortúzar', 'Agronomía', 'Parque Chas']
}


ahora insertaremos este valor en el dataframe

In [38]:
def obtener_cantidad_localidades(comuna):
    localidades = localidades_por_comuna.get(comuna)
    if localidades:
        return len(localidades)
    else:
        return None
    

comunas['Cantidad de Localidades'] = comunas['Comuna'].apply(obtener_cantidad_localidades)
comunas.head()

,Comuna,2022,Cantidad de Localidades
1,1,223.554,6
2,2,161.645,1
3,3,196.240,2
4,4,229.240,4
5,5,194.271,2


insertaremos una fila para la "comuna 0" que es el valor que le pusimos a todas aquellas comunas que tenian valor "SD"

In [40]:
new_row = {'Comuna': 0, '2022': 0, 'Cantidad de Localidades': 0}
comunas = pd.concat([pd.DataFrame(new_row, index=[0]), comunas]).reset_index(drop=True)


In [48]:
merged_df.to_csv("homicidios.csv",index=False)
lesiones.to_csv("lesiones.csv", index=False)
comunas.to_csv("comunas.csv", index = False)